In [13]:
# Imports
import sim
import solve
import rulesets

# Birth year
year_of_birth = 0

# Retirement goals
age_at_retirement = 0
age_at_death = 0
savings_at_death = 0
retirement_income = 0

# Initial conditions
initial_year = 0
initial_salary = 0
initial_savings_rrsp = 0
initial_savings_tfsa = 0

# Salary model parameters
# salary_plateau = the most you will ever earn - your salary will plateau when it reaches this figure
# salary_compound_rate = the rate at which your salary compounds per year, until it reaches the plateau. (Aka, your yearly raise.)
salary_plateau = 0
salary_compound_rate = 0.00

# Spending model parameters
# base_spending = 'Basic needs' spending, notionally the bare minimum yearly expenditure you need to get by. Expenditure above this is dubbed 'luxury spending'
# spending_luxury_compound_rate = the rate at which 'luxury spending' compounds. 
# These parameters can be tweaked to vary the distribution of early saving vs late saving
base_spending = 0
spending_luxury_compound_rate = 0.00

# Savings parameters
# initial_rrsp_allotment = the fraction of savings that is put into RRSP in the first year
# final_rrsp_allotment = the fraction of savings that is put into RRSP in the last year
# The actual split is a linear interpolation between the two values. This is intended as an exploratory model to find roughly optimal trends.
initial_rrsp_allotment = 0.00
final_rrsp_allotment = 0.00

# The interest rate earned by savings.
interest_rate = 0.00

#
simulation = sim.Simulation()
simulation.year_of_birth = year_of_birth
simulation.age_at_retirement = age_at_retirement
simulation.age_at_death = age_at_death
simulation.savings_at_death = savings_at_death
simulation.initial_year = initial_year
simulation.initial_salary = initial_salary
simulation._initial_savings_rrsp = initial_savings_rrsp
simulation.initial_savings_tfsa = initial_savings_tfsa

simulation.set_solver(solve.binary_solver)

career_rules, retirement_rules = rulesets.v1(
    salary_compound_rate=salary_compound_rate,
    salary_plateau=salary_plateau,
    base_spending=base_spending,
    spending_luxury_compound_rate=spending_luxury_compound_rate,
    initial_rrsp_allotment=initial_rrsp_allotment,
    final_rrsp_allotment=final_rrsp_allotment,
    initial_year=simulation.initial_year,
    year_of_retirement=simulation.year_of_retirement,
    year_of_death=simulation.year_of_death,
    retirement_income=retirement_income,
    rrsp_interest_rate=interest_rate,
    tfsa_interest_rate=interest_rate
)


In [5]:
#

In [ ]:
#

